In [1]:
import matplotlib.pyplot as plt
import numpy as np

import ipywidgets as widgets

from tqdm import tqdm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [83]:
def flux(rs, phis, As, pos, B, B0):
    posx, posy = pos

    N_r = rs.shape[0]
    N_s = phis.shape[0]

    rs2d = np.reshape(rs, (N_r, 1))
    phis2d = np.reshape(phis, (1, N_s))

    xs = posx + rs2d * np.cos(phis2d)
    ys = posy + rs2d * np.sin(phis2d)
    return (np.reshape(As, (N_r, 1))*B(xs, ys, B0)).sum()

In [104]:
def B_const(x, y, B0):
    return (x>0) * B0

def B_lin(x, y, B0):
    return (x>0) * 100*B0 * x

def B_exp(x, y, B0):
    return (x>0) * B0 * (1-np.exp(-50*x))

def B_2d(x, y, B0):
    return (x>0) * B0 * 100*B0*x * (1-y**2)

In [97]:
def Forces(rvecs, N_rings, N_sectors, Is, pos, B, B0):

    dls = np.diff(rvecs, axis=1)
    dls = np.append(dls, np.reshape(rvecs[:, 0]-rvecs[:, -1], (N_rings, 1, 3)), axis=1)

    x = (rvecs + dls/2 + pos)[:, :, 0]
    y = (rvecs + dls/2 + pos)[:, :, 1]
    
    Bvec = np.concatenate((np.zeros((2*N_rings, N_sectors)), B(x, y, B0)))
    Bvec = np.reshape(Bvec, (3, N_rings, N_sectors))
    Bvec = np.moveaxis(Bvec, 0, 2)

    Fs = (np.reshape(Is, (N_rings, 1, 1)) * np.cross(dls, Bvec)).sum(axis=1)

    return Fs

In [110]:
style = {'description_width': '150px'}
layout_slider = widgets.Layout(width='500px')

select_v0 = widgets.FloatSlider(value=0.2, min=-1.0, max=1.0, step=0.01, continuous_update=False, description='initial velocity', 
                                style=style, layout=layout_slider)
select_x0 = widgets.FloatSlider(value=-0.1, min=-0.5, max=0.5, step=0.01, continuous_update=False, description='initial position', 
                                style=style, layout=layout_slider)
select_tmax = widgets.FloatLogSlider(value=1, min=-2, max=2, step=0.05, continuous_update=False, description='time range', 
                                readout_format='.2f', style=style, layout=layout_slider)
select_rmax = widgets.FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, continuous_update=False, description='disc radius', 
                                readout_format='.2f', style=style, layout=layout_slider)
select_steps = widgets.IntSlider(value=50, min=10, max=500, step=10, continuous_update=False, description='number of steps', 
                                style=style, layout=layout_slider)
select_rings = widgets.IntSlider(value=25, min=10, max=200, step=1, continuous_update=False, description='number of rings', 
                                style=style, layout=layout_slider)
select_sectors = widgets.IntSlider(value=90, min=10, max=360, step=1, continuous_update=False, description='number of sectors', 
                                style=style, layout=layout_slider)

select_B0 = widgets.FloatSlider(value=0.1, min=0, max=1, step=0.01, continuous_update=False, description='characteristic B field', 
                                style=style, layout=layout_slider)

select_B = widgets.Dropdown(options=[('constant', B_const), 
                                     ('linear increase', B_lin), 
                                     ('exponential saturation', B_exp),
                                     ('2D field', B_2d)], 
                            description='magnetic field', style=style, layout=layout_slider)

select_pos = widgets.Checkbox(value=True, description='plot position')
select_vel = widgets.Checkbox(value=False, description='plot velocity')
select_force = widgets.Checkbox(value=False, description='plot force')
select_flux = widgets.Checkbox(value=False, description='plot flux')

In [113]:
@widgets.interact(v0=select_v0, x0=select_x0, tmax=select_tmax, rmax=select_rmax,
                  N_steps=select_steps, N_rings=select_rings, N_sectors=select_sectors,
                  B0=select_B0, B=select_B, plot_x=select_pos, plot_v=select_vel, plot_F=select_force, plot_flux=select_flux)
def plot(v0, x0, tmax, rmax, N_steps, N_rings, N_sectors, B0, B, plot_x, plot_v, plot_F, plot_flux):

    #prepare disc data
    dr = rmax/N_rings
    rs = np.linspace(dr, rmax, N_rings) - dr/2
    dphi = 2*np.pi/N_sectors
    phis = np.linspace(0, 2*np.pi-dphi, N_sectors)

    A_sectors = (rs+dr/2)**2 * np.pi/N_sectors
    A_sector_rings = np.diff(A_sectors)
    A_sector_rings = np.insert(A_sector_rings, 0, A_sectors[0])

    rhom = 8.92e3 # mass density of copper
    rho = 1.7e-8 # resistivity of copper
    h = 1e-3 # thickness of plate

    m = rhom * rmax*rmax*np.pi * h # mass of copper plate


    # define time and position arrays
    dt = tmax/N_steps
    t = np.linspace(0, tmax, N_steps)

    # define quantities for calculation of force
    x = np.zeros(N_steps) # define empty array for positions
    vx = np.zeros(N_steps) # define empty array for velocities
    Fx = np.zeros((N_steps, 3)) # define empty array for forces
    fluxes = np.zeros((N_steps, N_rings)) # define empty array for fluxes

    x[0] = x0 # initial position
    vx[0] = v0 # initial velocity
    fluxes[0] = np.array([flux(rs[0:i+1], phis, A_sector_rings[0:i+1], (x0, 0), B, B0) 
                      for i in range(N_rings)])

    Rs = rho*2*np.pi*rs/(dr*h) # resistance of rings

    # define vectors for segments
    rxs = [r * np.cos(phis) for r in rs]
    rys = [r * np.sin(phis) for r in rs]
    rzs = np.zeros((N_rings, N_sectors))

    r_vecs = np.moveaxis(np.array([rxs, rys, rzs]), 0, 2)

    # loop to calculate positions
    for i in range(N_steps-1):
        fluxes[i+1] = np.array([flux(rs[0:n+1], phis, A_sector_rings[0:n+1], (x[i], 0), B, B0) 
                          for n in range(N_rings)])

        Eis = -(fluxes[i+1]-fluxes[i])/dt # induced emf (Faraday's law)
        Is = Eis/Rs # calculate currents through rings

        Fs = Forces(r_vecs, N_rings, N_sectors, Is, (x[i], 0, 0), B, B0)
        Fx[i] = Fs.sum(axis=0)
        
        ax = Fx[i, 0]/m # acceleration in x direction
        
        vx[i+1] = vx[i] + ax*dt
        x[i+1] = x[i] + vx[i+1]*dt

    if plot_x:
        plt.plot(t, x, 'r', label='x vs t')
    if plot_v:
        plt.plot(t, vx, 'b', label='v vs t')
    if plot_F:
        plt.plot(t, -Fx[:, 0], 'g', label='|F| vs t')
    if plot_flux:
        plt.plot(t, fluxes.sum(axis=1)/dt, label=r'$\Phi$ vs t')
    
    plt.grid()
    plt.legend()

interactive(children=(FloatSlider(value=0.2, continuous_update=False, description='initial velocity', layout=L…